## Setup and imports

In [1]:
from pathlib import Path
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    InverseDynamicsController,
    MeshcatPoseSliders,
    Context,
    RigidTransform,
    RotationMatrix,
    Sphere,
    Rgba,
    LeafSystem,
    RollPitchYaw,
    MultibodyPlant,
    Parser,
    Solve,
    SolutionResult,
    AddMultibodyPlantSceneGraph,
    AddFrameTriadIllustration,
    FixedOffsetFrame,
    TrajectorySource,
    Trajectory,
    PiecewisePose,
    PiecewisePolynomial,
    ConstantValueSource,
    AbstractValue,
    JacobianWrtVariable
)
from pydrake.multibody import inverse_kinematics
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsIntegrator,
)
from pydrake.trajectories import PiecewisePolynomial


from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from manipulation.utils import RenderDiagram, running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad

import numpy as np

In [2]:
hammer_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="hammer">
    <static>false</static>

    <link name="hammer_link">
        <!-- Rough inertial properties (good enough for simulation) -->
        <inertial>
            <mass>1.0</mass>
            <inertia>
                <ixx>0.01</ixx>
                <iyy>0.01</iyy>
                <izz>0.01</izz>
                <ixy>0.0</ixy>
                <ixz>0.0</ixz>
                <iyz>0.0</iyz>
            </inertia>
        </inertial>

        <!-- Handle: long thin cylinder, centered along +z -->
        <visual name="handle_visual">
            <pose>0 0 0.125 0 0 0</pose>
            <geometry>
            <cylinder>
                <radius>0.015</radius>
                <length>0.25</length>
            </cylinder>
            </geometry>
            <material>
            <ambient>0.75 0.55 0.3 1</ambient>
            <diffuse>0.75 0.55 0.3 1</diffuse>
            </material>
        </visual>

        <collision name="handle_collision">
            <pose>0 0 0.125 0 0 0</pose>
            <geometry>
            <cylinder>
                <radius>0.01</radius>
                <length>0.25</length>
            </cylinder>
            </geometry>
        </collision>

        <!-- Head: short, fat cylinder at the top of the handle -->
        <visual name="head_visual">
            <pose>0 0 0.25 0 1.5708 0</pose>
            <geometry>
            <cylinder>
                <radius>0.05</radius>
                <length>0.12</length>
            </cylinder>
            </geometry>
            <material>
            <ambient>0.75 0.55 0.3 1</ambient>
            <diffuse>0.75 0.55 0.3 1</diffuse>
            </material>
        </visual>

        <collision name="head_collision">
            <pose>0 0 0.25 0 1.5708 0</pose>
            <geometry>
            <cylinder>
                <radius>0.05</radius>
                <length>0.12</length>
            </cylinder>
            </geometry>
        </collision>
    </link>
    <!-- frame for hammer head (center of head) -->
    <frame name="hammer_head" attached_to="hammer_link">
        <pose>0 0 0.25 0 1.5708 0</pose>
    </frame>
    <frame name="hammer_face" attached_to="hammer_link">
        <pose>0.06 0 0.25 0 1.5708 0</pose>
    </frame>
  </model>
</sdf>
"""

# Write to file
output_dir = Path("assets/")
with open(output_dir / "hammer.sdf", "w") as f:
  f.write(hammer_sdf)
hammer_sdf_filepath = f"{Path.cwd()}/assets/hammer.sdf"
print("Wrote hammer.sdf",hammer_sdf_filepath)

Wrote hammer.sdf /datasets/_deepnote_work/assets/hammer.sdf


In [3]:
grid_sdf = """<?xml version="1.6"?>
<sdf version="1.7">
  <model name="grid_board">

    <!-- 
      Single-link board + tabletop + cabinet.
      No internal joints → no possible loops.
    -->
    <link name="board">

      <inertial>
        <mass>8.0</mass>
        <inertia>
          <ixx>0.03</ixx><iyy>0.03</iyy><izz>0.03</izz>
          <ixy>0</ixy><ixz>0</ixz><iyz>0</iyz>
        </inertia>
      </inertial>

      <!-- Top plate (where holes and moles sit) -->
      <collision name="top_collision">
        <geometry>
          <box>
            <size>0.8 0.8 0.025</size>
          </box>
        </geometry>
      </collision>

      <visual name="top_visual">
        <geometry>
          <box>
            <size>0.8 0.8 0.025</size>
          </box>
        </geometry>
        <material>
          <ambient>0.1 0.4 0.1 1</ambient>   <!-- darker green -->
          <diffuse>0.2 0.7 0.2 1</diffuse>   <!-- bright green -->
        </material>
      </visual>

      <!-- Cabinet body beneath the tabletop -->
      <collision name="cabinet_collision">
        <geometry>
          <box>
            <size>0.6 0.6 0.45</size>
          </box>
        </geometry>
        <!-- Offset cabinet downward so its top meets the tabletop -->
        <pose>0 0 -0.2375 0 0 0</pose>
      </collision>

      <visual name="cabinet_visual">
        <geometry>
          <box>
            <size>0.6 0.6 0.45</size>
          </box>
        </geometry>
        <material>
          <ambient>0.6 0.6 0.65 1</ambient>
          <diffuse>0.75 0.75 0.8 1</diffuse>
        </material>
        <pose>0 0 -0.2375 0 0 0</pose>
      </visual>

    </link>

    <static>false</static>
  </model>
</sdf>
"""

# Write to file
output_dir = Path("assets/")
with open(output_dir / "grid.sdf", "w") as f:
  f.write(grid_sdf)
grid_sdf_filepath = f"{Path.cwd()}/assets/grid.sdf"
print("Wrote grid.sdf",grid_sdf_filepath)

Wrote grid.sdf /datasets/_deepnote_work/assets/grid.sdf


## Perception

In [4]:
from pydrake.all import LeafSystem, BasicVector, AbstractValue, RigidTransform

class DummyPerception(LeafSystem):
    def __init__(self, plant, mole_indices):
        """
        Args:
            plant: The MultibodyPlant (shared reference).
            mole_indices: List of ModelInstanceIndex for the moles.
        """
        LeafSystem.__init__(self)
        self._plant = plant
        self._context = plant.CreateDefaultContext() # Internal context for calculation
        self._mole_indices = mole_indices
        
        # Input: the full state of the plant (positions + velocities of everything)
        self.DeclareVectorInputPort(
            "plant_state", 
            BasicVector(plant.num_multibody_states())
        )

        # Output: AbstractValue (python dictionary {mole_index: RigidTransform of mole world pose})
        self.DeclareAbstractOutputPort(
            "mole_poses",
            lambda: AbstractValue.Make({}),
            self.CalcMolePoses
        )

    def CalcMolePoses(self, context, output):
        # Update internal plant context with current simulation state
        state_input = self.EvalVectorInput(context, 0).get_value()
        self._plant.SetPositionsAndVelocities(self._context, state_input)
        
        mole_data = {}
        # Compute poses for each mole
        for i, mole_idx in enumerate(self._mole_indices):
            mole_head = self._plant.GetBodyByName("mole", mole_idx)
            X_WMole = self._plant.EvalBodyPoseInWorld(self._context, mole_head)
            mole_data[i] = X_WMole
            
        output.set_value(mole_data)

## Prehit Planner

In [5]:
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.multibody.tree import ModelInstanceIndex

class IiwaMotionPlanner:
    def __init__(self, plant, iiwa_model_instance, hammer_frame_name="hammer_face"):
        self.plant = plant
        # We create a specific context for planning so we don't mess with simulation state
        self.context = plant.CreateDefaultContext()
        self.iiwa = iiwa_model_instance
        
        # Cache frames
        self.world_frame = plant.world_frame()
        self.hammer_model = plant.GetModelInstanceByName("hammer")
        self.hammer_face_frame = plant.GetFrameByName(hammer_frame_name, self.hammer_model)
        self.iiwa_link7_frame = plant.GetFrameByName("iiwa_link_7", self.iiwa)

    def get_prehit_pose(self, X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
        """
        Calculates the pre-hit pose for hammer face X_WH_prehit based on the object pose X_WO.
        Returns:
            X_WH_prehit: Pose of the hammer face in World
            X_WL7_prehit: Pose of Link 7 in World (useful for debugging/seeding)
        """
        # Fixed offset from Object to Hammer Face (Your logic)
        p_OH = np.array([0.0, 0, 0.2])
        R_OH = RotationMatrix.MakeYRotation(np.pi) @ RotationMatrix.MakeZRotation(np.pi/2)
        X_OH = RigidTransform(R_OH, p_OH) 
        
        X_WH_prehit = X_WO @ X_OH
        
        # Calculate X_HL7 (Hammer Face to Link 7)
        # We can get this from the plant's default context since it's a rigid weld
        X_WL7_default = self.plant.CalcRelativeTransform(
            self.context, self.world_frame, self.iiwa_link7_frame
        )
        X_WH_default = self.plant.CalcRelativeTransform(
            self.context, self.world_frame, self.hammer_face_frame
        )
        # X_HL7 = (X_WH)^-1 @ X_WL7
        X_HL7 = X_WH_default.inverse() @ X_WL7_default
        
        X_WL7_prehit = X_WH_prehit @ X_HL7
        
        return X_WH_prehit, X_WL7_prehit

    def plan_prehit(self, q_current, target_pose_world):
        """
        High-level function: Takes current joints and target object pose,
        returns a trajectory to the pre-hit pose.
        """
        # Get desired prehit pose for hammer face X_WH_prehit given target mole pose
        X_WH_prehit, X_WL7_seed = self.get_prehit_pose(target_pose_world)
        
        # Solve IK for the target iiwa prehit config q_goal
        q_goal, success = self.solve_ik(X_WH_prehit, q_guess=q_current)
        
        if not success:
            print(f"IK failed to find a prehit config for target mole {target_pose_world}")
            return None

        # Generate Trajectory
        # Simple straight line in joint space
        traj = self.make_joint_space_position_trajectory([q_current, q_goal], duration=2.0)
        return traj

    def solve_ik(self, X_WH, q_guess=None, pos_tol=1e-3, theta_bound=1e-2):
        ik = InverseKinematics(self.plant, self.context)
        prog = ik.prog()
        # Positions of all objects in plant
        q_current_full = self.plant.GetPositions(self.context)
        q_vars = ik.q()
        # if q_guess is None:
        #     q_guess = self.plant.GetPositions(self.context, self.iiwa)

        R_WH_desired = X_WH.rotation()
        p_WH_desired = X_WH.translation()

        # Position Constraint
        ik.AddPositionConstraint(
            frameA=self.world_frame,
            p_BQ=np.zeros(3),
            frameB=self.hammer_face_frame,
            p_AQ_lower=p_WH_desired - pos_tol,
            p_AQ_upper=p_WH_desired + pos_tol,
        )

        # Orientation Constraint
        ik.AddOrientationConstraint(
            frameAbar=self.world_frame,
            R_AbarA=RotationMatrix(), 
            frameBbar=self.hammer_face_frame,
            R_BbarB=R_WH_desired,
            theta_bound=theta_bound,
        )

        # IMPORTANT: Lock all non-IIWA joints
        # If we don't do this, the solver will move the moles to satisfy constraints.
        
        # Iterate over all models to find which indices in 'q' belong to "Not IIWA"
        indices_to_lock = []
        world_instance = self.plant.GetModelInstanceByName("WorldModelInstance")
        iiwa_instance = self.iiwa

        for i in range(self.plant.num_model_instances()):
            model_instance = ModelInstanceIndex(i)
            if model_instance in (iiwa_instance, world_instance):
                continue  # skip iiwa and world
            # For each non-iiwa instance, get all joint indices
            for joint_idx in self.plant.GetJointIndices(model_instance):
                joint = self.plant.get_joint(joint_idx)
                # Get the start index in the q vector for this joint
                start = joint.position_start()
                num = joint.num_positions()
                indices_to_lock.extend(range(start, start + num))
    
        # Apply the lock constraint
        if indices_to_lock:
            indices_to_lock = np.array(indices_to_lock, dtype=int)
            prog.AddBoundingBoxConstraint(
                q_current_full[indices_to_lock], # Lower bound (current val)
                q_current_full[indices_to_lock], # Upper bound (current val)
                q_vars[indices_to_lock]          # The decision variables
            )

        # Solve 
        prog.SetInitialGuess(q_vars, q_current_full)
        result = Solve(prog)
        
        # Extract only the iiwa solution
        q_sol_full = result.GetSolution(q_vars)
        self.plant.SetPositions(self.context, q_sol_full)
        q_sol_iiwa = self.plant.GetPositions(self.context, self.iiwa)
        
        # Return solution and success
        if result.get_solution_result() != SolutionResult.kSolutionFound:
            return q_sol_iiwa, False
        return q_sol_iiwa, True

    def make_joint_space_position_trajectory(self, path, duration=2.0):
        times = np.linspace(0, duration, len(path))
        Q = np.column_stack(path)
        return PiecewisePolynomial.FirstOrderHold(times, Q)

## Force Control

In [6]:
from pydrake.multibody.plant import ContactResults

class HammerContactForce(LeafSystem):
    """
    Reads ContactResults from the plant and outputs the scalar contact force
    on the hammer body along n_hat (in world frame).
    """
    def __init__(self, plant, hammer_body_index, n_hat):
        super().__init__()
        self._plant = plant
        self._hammer_body_index = hammer_body_index
        # Normalize n_hat
        self._n_hat = np.array(n_hat) / np.linalg.norm(n_hat)

        # Abstract input: ContactResults
        self.DeclareAbstractInputPort(
            "contact_results",
            AbstractValue.Make(ContactResults())
        )

        # Scalar output: F_meas
        self.DeclareVectorOutputPort(
            "F_meas", BasicVector(1),
            self.CalcOutput
        )

    def CalcOutput(self, context, output):
        contact_results = self.EvalAbstractInput(context, 0).get_value()
        F_W = np.zeros(3)

        # Sum up all forces acting on the hammer
        for i in range(contact_results.num_point_pair_contacts()):
            info = contact_results.point_pair_contact_info(i)
            f_Bc_W = info.contact_force()
            bodyA = info.bodyA_index()
            bodyB = info.bodyB_index()

            if bodyA == self._hammer_body_index:
                F_W -= f_Bc_W    # Force on Hammer is negative
            if bodyB == self._hammer_body_index:
                F_W += f_Bc_W    # Force on Hammer is positive

        F_meas = float(np.dot(self._n_hat, F_W))
        output.SetAtIndex(0, F_meas)

class HitAdmittanceController:
    """
    'Virtual Hammer' Controller with 1D Admittance + joint-space posture spring.
        1D admittance along n_hat using ALL joints to realize the motion; 
        also apply posture spring toward q_prehit.
    
    Physics:
        M*a + D*v + K*s = F_des - F_meas
        
    Kinematics:
        q_cmd = q_anchor + pinv(J) * (n_hat * s)
    """
    def __init__(self, M=0.5, D=10.0, K=0.0, dt=0.01, n_hat=[0,0,-1]):
        self.M = M
        self.D = D
        self.K = K
        self.dt = dt
        self.n_hat = np.array(n_hat) / np.linalg.norm(n_hat)

    def compute_next_state(self, s, s_dot, F_des, F_meas):
        """
        Performs one step of Euler Integration for the admittance physics.
        Returns: (s_new, s_dot_new)
        """
        # F_net opposes motion (F_meas is magnitude against n_hat)
        F_err = F_des - F_meas
        
        # Dynamics: a = (F_err - D*v - K*s) / M
        s_ddot = (F_err - self.D * s_dot - self.K * s) / self.M
        
        # Euler Integration
        s_new = s + s_dot * self.dt
        s_dot_new = s_dot + s_ddot * self.dt
        
        return s_new, s_dot_new

    def compute_q_cmd(self, q_anchor, s, J):
        """
        Converts the admittance scalar 's' into a Joint Configuration command.
        
        Args:
            q_anchor: The joint configuration at the start of the hit (7,)
            s: The current admittance displacement (scalar)
            J: The Jacobian matrix at the current configuration (6,7) or (3,7)
        """
        # Task space displacement vector
        dx = self.n_hat * s
        
        # Inverse Kinematics (Differential)
        # dq = J# * dx
        dq = np.linalg.pinv(J) @ dx\
        
        return q_anchor + dq

## State Machine Planner
Finite State Machine (FSM) named BonkBotBrain

In [7]:
import numpy as np
from pydrake.all import (
    LeafSystem, BasicVector, AbstractValue, RigidTransform, EventStatus
)
class FSMState:
    WAIT = 0
    PLANNING = 1
    APPROACH = 2
    HIT = 3
    RECOVER = 4
    GO_HOME = 5


class BonkBotBrain(LeafSystem):
    def __init__(self, plant, iiwa_model_instance, mole_instances):
        LeafSystem.__init__(self)
        
        self.plant = plant
        self.iiwa = iiwa_model_instance
        
        # Configuration
        self.q_home = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0]) # home iiwa pose
        self.mole_threshold = 0.08
        self.F_des = 15.0 # Desired Force
        self.dt = 0.01 # 100 Hz Control Loop

        # Initialize Helper Classes
        self.motion_planner = IiwaMotionPlanner(plant, iiwa_model_instance)
        self.admittance = HitAdmittanceController(
            M=1.0, D=40.0, K=0.0, dt=self.dt, n_hat=[0, 0, -1.0]
        )
        
        # Inputs 
        self._iiwa_pos_index = self.DeclareVectorInputPort("iiwa_position", BasicVector(7)).get_index()
        self._iiwa_vel_index = self.DeclareVectorInputPort("iiwa_velocity", BasicVector(7)).get_index()
        self._perception_index = self.DeclareAbstractInputPort(   # (Dictionary of mole id: RigidTransform)
                                    "mole_poses", AbstractValue.Make({})).get_index()
        self._force_index = self.DeclareVectorInputPort("F_meas", BasicVector(1)).get_index()
        
        # Output: iiwa position command
        self.DeclareVectorOutputPort("iiwa_position_command", BasicVector(7), self.CalcIiwaCommand)

        # States
        self._fsm_state_index = self.DeclareAbstractState(AbstractValue.Make(FSMState.WAIT))
        self._traj_index = self.DeclareAbstractState(AbstractValue.Make(None))
        # Logic [target_mole_idx, action_start_time, tick_count]
        self._logic_state_index = self.DeclareDiscreteState(3) 
        # Admittance [s, s_dot, q_anchor(7)] = [displacement along n_hat, velocity along n_hat, iiwa joint positions]
        self._admittance_state_index = self.DeclareDiscreteState(2 + 7)

        # Schedule
        self.DeclarePeriodicUnrestrictedUpdateEvent(
            period_sec=self.dt,
            offset_sec=0.0,
            update=self.UpdateFSM
        )
    
    # --- HELPER FUNCTIONS ---
    
    def get_iiwa_state(self, context):
        """Helper to combine pos and vel into a 14-vector for calculations"""
        q = self.EvalVectorInput(context, self._iiwa_pos_index).get_value()
        v = self.EvalVectorInput(context, self._iiwa_vel_index).get_value()
        return q, v

    def get_active_mole(self, mole_poses):
        """
        Helper that returns index of the first mole that is up (above threshold).
        "First" here just means first mole index, not accounting for time for now.
        Returns -1 if none found.
        """
        heights = {i: p.translation()[2] for i, p in mole_poses.items()}
        for idx, pose in mole_poses.items():
            # if z position of mole greater than threshold
            if pose.translation()[2] > self.mole_threshold:
                return idx
        return -1
    
    def check_target_mole_valid(self, logic_state_val, mole_poses):
        """
        Helper to check whether current target mole is still up.
        Returns tuple (is_valid, target_idx, target_pose)
        """
        # Get current target from logic state
        target_idx = int(logic_state_val[0]) # mole index
        target_pose = mole_poses[target_idx]
        # Check mole existence and height
        target_is_valid = (target_pose is not None and 
                            target_pose.translation()[2] > self.mole_threshold)
        return target_is_valid, target_idx, target_pose

    def start_trajectory_action(self, new_fsm, new_traj, new_logic, 
                                next_state, trajectory, current_time):
        """
        Helper to consolidate updating state, trajectory, and start time.
        """
        new_traj.set_value(trajectory)
        new_logic.get_mutable_value()[1] = current_time
        new_fsm.set_value(next_state)

    def try_attack(self, active_mole, new_fsm, new_logic):
        """
        Helper to check for a mole target and transition to attack, esp while in motion. 
            (BonkBot follows ABC: Always Be Checking for up moles to attack.)
        Returns False if no valid targets.
        """
        if active_mole >= 0:
            print(f"[Brain] DETECTED MOLE {active_mole}! ATTACK! Switch to PLANNING.")
            new_logic.get_mutable_value()[0] = active_mole 
            new_fsm.set_value(FSMState.PLANNING)
            return True
        return False
        
    # --- MAIN LOGIC ---

    def CalcIiwaCommand(self, context, output):
        """Continuous control loop that evaluates trajectory."""
        current_time = context.get_time()
        fsm_state = context.get_abstract_state(self._fsm_state_index).get_value()
        logic_state = context.get_discrete_state(self._logic_state_index).get_value()
        q_current, _ = self.get_iiwa_state(context)
        
        # Default to holding current position (WAIT, PLANNING)
        q_cmd = q_current 
        
        # If hitting
        if fsm_state == FSMState.HIT:
            adm_state = context.get_discrete_state(self._admittance_state_index).get_value()
            s = adm_state[0]
            q_anchor = adm_state[2:]
            
            # Calculate Jacobian live
            context_plant = self.motion_planner.context
            self.plant.SetPositions(context_plant, self.iiwa, q_current)
            J_spatial = self.plant.CalcJacobianTranslationalVelocity(
                context_plant, JacobianWrtVariable.kQDot,
                self.motion_planner.hammer_face_frame, np.zeros(3),
                self.plant.world_frame(), self.plant.world_frame()
            )
            # Slice for IIWA (first 7 columns)
            J = J_spatial[:, 0:7] 

            q_cmd = self.admittance.compute_q_cmd(q_anchor, s, J)
        
        # If in other moving state, execute trajectory
        elif fsm_state in {FSMState.APPROACH, FSMState.RECOVER, FSMState.GO_HOME}:
            traj = context.get_abstract_state(self._traj_index).get_value()
            if traj:
                start_time = logic_state[1]
                t_rel = current_time - start_time
                q_cmd = traj.value(min(t_rel, traj.end_time())).flatten()
                
        output.SetFromVector(q_cmd)


    def UpdateFSM(self, context, discrete_state):
        """
        Main BonkBot Brain Logic
        """
        current_time = context.get_time()
        
        # Get Mutable States
        new_fsm_state = discrete_state.get_mutable_abstract_state(self._fsm_state_index)
        new_traj = discrete_state.get_mutable_abstract_state(self._traj_index)
        new_logic = discrete_state.get_mutable_discrete_state(self._logic_state_index)
        new_adm = discrete_state.get_mutable_discrete_state(self._admittance_state_index)
        
        # Current Values (read-only)
        current_fsm = context.get_abstract_state(self._fsm_state_index).get_value()
        mole_poses = self.EvalAbstractInput(context, self._perception_index).get_value()
        logic_val = context.get_discrete_state(self._logic_state_index).get_value() # [target_mole_idx, action_start_time]
        tick_count = int(logic_val[2])
        q_current, v_current = self.get_iiwa_state(context)
        F_meas_scalar = self.EvalVectorInput(context, self._force_index).get_value()[0]

        if current_time % 1 == 0:
            print(f"[DEBUG t={current_time:.2f}] State: {current_fsm}")

        # --- FSM LOGIC ---

        # FAST LOOP (100Hz): FORCE CONTROL
        if current_fsm == FSMState.HIT:
            s = new_adm.get_value()[0]
            s_dot = new_adm.get_value()[1]
            
            # Calculate next s, s_dot
            s_new, s_dot_new = self.admittance.compute_next_state(
                s, s_dot, self.F_des, F_meas_scalar
            )
            new_adm.SetAtIndex(0, s_new)
            new_adm.SetAtIndex(1, s_dot_new)
            
            # Hit Timeout Check
            start_time = logic_val[1]
            if current_time - start_time > 0.5:
                print("[Brain] Hit timeout/complete. Switch to RECOVER.")
                # Recover trajectory is just reverse out from where we currently are
                q_anchor = new_adm.get_value()[2:]
                recover_traj = self.motion_planner.make_joint_space_position_trajectory(
                    [q_current, q_anchor], duration=0.5
                )
                self.start_trajectory_action(new_fsm_state, new_traj, new_logic,
                                             FSMState.RECOVER, recover_traj, current_time)
                
        # SLOW LOOP (50Hz): OTHER STATES
        else:
            # Enforce slower rates for non-hit states
            new_logic.SetAtIndex(2, tick_count + 1)
            if tick_count % 2 != 0:
                return EventStatus.Succeeded()

            # Check Perception
            active_mole = self.get_active_mole(mole_poses) # most recent active mole
            
            if current_fsm == FSMState.WAIT:
                # MOLE CHECK: yo mole, you up?
                if not self.try_attack(active_mole, new_fsm_state, new_logic):
                    if current_time % 1 == 0:
                        print("[Brain] No mole is up.") # ;-;

            elif current_fsm == FSMState.PLANNING:            
                # MOLE CHECK: you still up?
                valid, target_idx, target_pose = self.check_target_mole_valid(logic_val, mole_poses)
                if not valid:
                    # If current mole gone, try switch to new mole.
                    if active_mole >= 0:
                        print(f"[Brain] Mole {target_idx} gone before planning. Retarget Mole {active_mole}.")
                        new_logic.get_mutable_value()[0] = active_mole
                    else:
                        print(f"[Brain] Mole {target_idx} gone before planning. Switch to WAIT.")
                        new_fsm_state.set_value(FSMState.WAIT)
                        return EventStatus.Succeeded()
                
                # Calculate path from current pose to target prehit pose
                print(f"[Brain] Planning pre-hit path to mole {target_idx} at {target_pose.translation()}")
                traj = self.motion_planner.plan_prehit(q_current, target_pose)
                
                if traj is not None:
                    print("[Brain] Prehit path found. Switch to APPROACH.")
                    self.start_trajectory_action(new_fsm_state, new_traj, new_logic,
                                                FSMState.APPROACH, traj, current_time)
                else:
                    print("[Brain] Prehit plan failed (IK). Return to WAIT.")
                    new_fsm_state.set_value(FSMState.WAIT)

            elif current_fsm == FSMState.APPROACH:
                # MOLE CHECK: you still up?
                valid, target_idx, _ = self.check_target_mole_valid(logic_val, mole_poses)
                if not valid:
                    # If current mole gone, try switch to new mole.
                    if active_mole >= 0:
                        print(f"[Brain] Mole {target_idx} gone while approaching. Retarget Mole {active_mole}.")
                        new_logic.get_mutable_value()[0] = active_mole
                        new_fsm_state.set_value(FSMState.PLANNING)
                    else:
                        print(f"[Brain] Mole {target_idx} gone while approaching. Switch to WAIT.")
                        new_fsm_state.set_value(FSMState.WAIT)
                    return EventStatus.Succeeded()
                
                traj = context.get_abstract_state(self._traj_index).get_value()
                duration = current_time - logic_val[1] # action start time
                if current_time % 1 == 0:
                    print(f"[Brain] Approaching... ({duration:.2f}/{traj.end_time():.2f}s)")

                # Check if prehit complete.
                if duration >= traj.end_time():
                    print("[Brain] Pre-hit reached. Switching to ADMITTANCE HIT.")
                    # Set Admittance state
                    # Initial Admittance: s=0, v=0.2 (impact velocity)
                    new_adm.SetAtIndex(0, 0.0)
                    new_adm.SetAtIndex(1, 0.2) 
                    # Set Anchor Pose
                    for i in range(7):
                        new_adm.SetAtIndex(2+i, q_current[i])
                    new_fsm_state.set_value(FSMState.HIT)
                    new_logic.SetAtIndex(1, current_time)

            elif current_fsm == FSMState.RECOVER:
                traj = context.get_abstract_state(self._traj_index).get_value()
                duration = current_time - logic_val[1]

                # Check if recover complete.
                if duration >= traj.end_time():
                    # MOLE CHECK: you up? 
                    if self.try_attack(active_mole, new_fsm_state, new_logic):
                        return EventStatus.Succeeded()
                    print("[Brain] Recovered. No targets. Going HOME.")
                    home_traj = self.motion_planner.make_joint_space_position_trajectory(
                        [q_current, self.q_home], duration=2.0
                    )
                    self.start_trajectory_action(new_fsm_state, new_traj, new_logic,
                                                FSMState.GO_HOME, home_traj, current_time)
                    
            elif current_fsm == FSMState.GO_HOME:
                # MOLE CHECK: you up?
                if self.try_attack(active_mole, new_fsm_state, new_logic):
                    return EventStatus.Succeeded()

                traj = context.get_abstract_state(self._traj_index).get_value()
                duration = current_time - logic_val[1]
                
                # Check if go home complete.
                if duration >= traj.end_time():
                    print("[Brain] Arrived Home. Waiting.")
                    new_fsm_state.set_value(FSMState.WAIT)

        return EventStatus.Succeeded()

## Mole Scheduler

In [8]:
import numpy as np
import random
from pydrake.all import (
    LeafSystem, BasicVector, AbstractValue, EventStatus, ContactResults,
    ModelInstanceIndex
)

class MoleController(LeafSystem):
    """
    Manages the logic for popping moles up and down.
    
    Hit Detection Criteria:
    - Active mole must be in contact with Hammer.
    - Active mole must be pushed down below 'hit_threshold' (2.5 cm).
    """
    def __init__(self, plant, hammer_body_index, mole_model_indices, 
                    switch_time_range=(1.5, 3.0), random_switch_time=False, multi_mode=False):
        LeafSystem.__init__(self)
        
        self.plant = plant
        self.context_plant = plant.CreateDefaultContext()
        self.hammer_idx = hammer_body_index
        self.mole_models = mole_model_indices 
        self.num_moles = len(mole_model_indices)
        self.min_time, self.max_time = switch_time_range
        self.random_switch_time = random_switch_time
        self.multi_mode = multi_mode
        
        # Get mole body indices for contact check
        self.mole_body_indices = []
        for model_idx in self.mole_models:
            body = plant.GetBodyByName("mole", model_idx)
            self.mole_body_indices.append(body.index())
        
        # Thresholds
        self.hit_height_threshold = 0.025 # Must be pushed down to 2.5cm
        self.spawn_grace_period = 0.5 # Ignore low height for 0.5s after spawn

        # Input Ports
        # Contact Results (to check collisions)
        self._contact_input_index = self.DeclareAbstractInputPort(
                                        "contact_results",
                                        AbstractValue.Make(ContactResults())).get_index()
        # Plant State (to check heights)
        self._state_input_index = self.DeclareVectorInputPort(
                                        "plant_state",
                                        BasicVector(plant.num_multibody_states())).get_index()
        # Output Port
        self.mole_ports = []
        for i in range(self.num_moles):
            port = self.DeclareVectorOutputPort(
                        f"mole_{i}_setpoint", 
                        BasicVector(2), 
                        lambda context, output, idx=i: self.CalcMoleCommand(context, output, idx)
                    )
            self.mole_ports.append(port)
        
        # States
        # Timer State: [next_switch_time]
        self._timer_index = self.DeclareDiscreteState(1)
        # Mole States: vector with len of num_moles, 1 = Active/Up, 0 = Inactive/Down
        self._moles_index = self.DeclareDiscreteState(self.num_moles)
        # Activation Times: track when each mole became active for spawn_grace_period
        self._activation_times_index = self.DeclareDiscreteState(self.num_moles)
        
        # Schedule
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=0.02, 
            offset_sec=0.0, 
            update=self.UpdateLogic
        )

    def CalcMoleCommand(self, context, output, mole_idx):
        mole_states = context.get_discrete_state(self._moles_index).get_value()
        is_active = (mole_states[mole_idx] > 0.5) # check if mole up
        if is_active:
            q_des = 0.12 # Pop up
        else:
            q_des = 0.0  # Stay down
        v_des = 0.0
        output.SetFromVector([q_des, v_des])

    def handle_hits(self, moles_vec, activation_vec, timer_vec, current_time):
        """
        Checks height only (with time buffer) to register hits.
        """
        for i in range(self.num_moles):
            # Only check active moles
            if moles_vec.get_value()[i] > 0.5:
                # Check Grace Period
                # (Mole starts at 0.0 height, so we must wait for it to rise 
                # before we start checking if it's been pushed down)
                spawn_time = activation_vec.get_value()[i]
                if (current_time - spawn_time) < self.spawn_grace_period:
                    continue
                # Check Height: is mole pushed down?
                model_idx = self.mole_models[i]
                q_mole = self.plant.GetPositions(self.context_plant, model_idx)[0]
                is_pushed_down = q_mole < self.hit_height_threshold
                if is_pushed_down:
                    print(f"[BONK] Mole {i} bonked! (Height: {q_mole:.3f}m)")
                    # Deactivate Mole
                    moles_vec.SetAtIndex(i, 0.0) 
                    # Reset Timer (pause briefly to prevent mole from popping up immediately)
                    new_spawn_time = current_time + 1.0
                    timer_vec.SetAtIndex(0, new_spawn_time)

        
    def UpdateLogic(self, context, discrete_state):
        current_time = context.get_time()
        
        # Mutable State
        timer_vec = discrete_state.get_mutable_vector(self._timer_index)
        moles_vec = discrete_state.get_mutable_vector(self._moles_index)
        activation_vec = discrete_state.get_mutable_vector(self._activation_times_index)

        # Get the next switch, aka next time a mole will move
        next_switch = timer_vec.get_value()[0]
        if next_switch == 0.0:
            if self.random_switch_time:
                step = random.uniform(self.min_time, self.max_time)
            else:
                step = self.max_time
            next_switch = current_time + step
            timer_vec.SetAtIndex(0, next_switch)
        
        # Update plant contexts
        plant_state = self.EvalVectorInput(context, self._state_input_index).get_value()
        self.plant.SetPositionsAndVelocities(self.context_plant, plant_state)
        contact_results = self.EvalAbstractInput(context, self._contact_input_index).get_value()
        
        # Hit Detection
        self.handle_hits(moles_vec, activation_vec, timer_vec, current_time)
        
        # Pick next mole action
        if current_time >= next_switch:
            # Pick random mole index (this is the next mole that can move)
            target_idx = random.randint(0, self.num_moles - 1)
            if self.multi_mode:
                # MULTI MODE: Toggle this specific mole
                current_val = moles_vec.get_value()[target_idx]
                new_val = 1.0 - current_val # flip state of the selected mole
                moles_vec.SetAtIndex(target_idx, new_val)
                if new_val > 0.5:
                    activation_vec.SetAtIndex(target_idx, current_time)
                    state_str = "UP"
                else:
                    state_str = "DOWN"
                print(f"[Mole] Mole {target_idx} -> {state_str} (Multi Mode)")
            else:
                # SINGLE MODE: Clear all, set one
                moles_vec.SetFromVector(np.zeros(self.num_moles))
                moles_vec.SetAtIndex(target_idx, 1.0)
                activation_vec.SetAtIndex(target_idx, current_time)
                print(f"[Mole] Mole {target_idx} UP (Single Mode)")
            
            # Reset Timer
            if self.random_switch_time:
                duration = random.uniform(self.min_time, self.max_time)
            else:
                duration = self.max_time
            timer_vec.SetAtIndex(0, current_time + duration)
            
        return EventStatus.Succeeded()

## Simulation

In [9]:
def create_mole_sdf_file(filename="mole.sdf"):
    sdf_content = """<?xml version="1.6"?>
<sdf xmlns:drake="http://drake.mit.edu" version="1.7">
  <model name="mole_unit">

    <!-- Socket welded to the board -->
    <link name="socket">
      <inertial>
        <mass>0.5</mass>
        <inertia>
          <ixx>1e-4</ixx><iyy>1e-4</iyy><izz>1e-4</izz>
        </inertia>
      </inertial>

      <!-- Decorative ring around mole hole -->
      <visual name="socket_ring">
        <pose>0 0 0.005 0 0 0</pose>
        <geometry>
          <cylinder>
            <radius>0.07</radius>
            <length>0.004</length>
          </cylinder>
        </geometry>
        <material>
          <ambient>0.1 0.1 0.1 1</ambient>
          <diffuse>0.2 0.2 0.2 1</diffuse>
        </material>
      </visual>
      
    </link>

    <!-- Moving mole -->
    <link name="mole">
        <inertial>
            <mass>0.2</mass>
            <inertia>
            <ixx>1e-4</ixx><iyy>1e-4</iyy><izz>1e-4</izz>
            </inertia>
        </inertial>

        <!-- Cylinder piece mole body -->
        <visual name="body">
            <pose>0 0 -0.045 0 0 0</pose>  <!-- top at z = 0.03, bottom at z = -0.17 -->
            <geometry>
            <cylinder>
                <radius>0.05</radius>
                <length>0.2</length>
            </cylinder>
            </geometry>
            <material>
                <diffuse>0.70 0.48 0.27 1</diffuse>
                <ambient>0.28 0.17 0.08 1</ambient>
                <specular>0.05 0.05 0.05 1</specular>
            </material>
        </visual>

        <!-- Rounded cap on top of the body (visual-only hemisphere) -->
        <visual name="head_cap">
            <!-- Centered at the top of the cylinder: center at z = 0.03; top at 0.08 -->
            <pose>0 0 0.055 0 0 0</pose>
            <geometry>
            <sphere>
                <radius>0.05</radius>
            </sphere>
            </geometry>
            <material>
                <diffuse>0.70 0.48 0.27 1</diffuse>
                <ambient>0.28 0.17 0.08 1</ambient>
                <specular>0.05 0.05 0.05 1</specular>
            </material>
        </visual>

        <!-- Collision: just a cylinder -->
        <collision name="body_collision">
            <pose>0 0 0.055 0 0 0</pose>
            <geometry>
            <cylinder>
                <radius>0.05</radius>
                <length>0.10</length>
            </cylinder>
            </geometry>
        </collision>

        <!-- Eyes on rounded top -->
        <visual name="eye_left">
            <!-- x = left/right, y = "forward", z = up -->
            <pose>-0.024 0.045 0.06 0 0 0</pose>
            <geometry>
            <sphere>
                <radius>0.0075</radius>
            </sphere>
            </geometry>
            <material>
            <diffuse>0 0 0 1</diffuse>
            <ambient>0 0 0 1</ambient>
            </material>
        </visual>

        <visual name="eye_right">
            <pose>0.024 0.045 0.06 0 0 0</pose>
            <geometry>
            <sphere>
                <radius>0.0075</radius>
            </sphere>
            </geometry>
            <material>
            <diffuse>0 0 0 1</diffuse>
            <ambient>0 0 0 1</ambient>
            </material>
        </visual>

        <!-- Left cheek blush -->
        <visual name="blush_left">
            <!-- x: left/right, y: forward, z: up -->
            <pose>-0.0275 0.04 0.045 0 0 0</pose>
            <geometry>
            <drake:ellipsoid>
                <!-- half-axes (meters): a=x, b=y, c=z -->
                <a>0.008</a>   <!-- width -->
                <b>0.007</b>   <!-- height -->
                <c>0.004</c>   <!-- very thin (almost a decal) -->
            </drake:ellipsoid>
            </geometry>
            <material>
                <ambient>0.7 0.5 0.6 1</ambient>
                <diffuse>1.0 0.7 0.8 1</diffuse>   <!-- light pink -->
                <specular>0.03 0.03 0.03 1</specular> <!-- matte-ish -->
            </material>
        </visual>

        <!-- Right cheek blush -->
        <visual name="blush_right">
            <pose>0.0275 0.04 0.045 0 0 0</pose>
            <geometry>
            <drake:ellipsoid>
                <a>0.008</a>
                <b>0.007</b>
                <c>0.004</c>
            </drake:ellipsoid>
            </geometry>
            <material>
                <ambient>0.7 0.5 0.6 1</ambient>
                <diffuse>1.0 0.7 0.8 1</diffuse>
                <specular>0.03 0.03 0.03 1</specular>
            </material>
        </visual>

        <!-- Little brown nose, slightly in front and below eyes -->
        <visual name="nose">
            <pose>0 0.052 0.055 0 0 0</pose>
            <geometry>
            <sphere>
                <radius>0.008</radius>
            </sphere>
            </geometry>
            <material>
                <diffuse>0.5 0.3 0.2 1</diffuse>
                <ambient>0.3 0.2 0.15 1</ambient>
            </material>
        </visual>

        <!-- Buck teeth: two thin white rectangles just under the nose -->
        <visual name="tooth_left">
            <!-- Slightly left, slightly forward, below nose -->
            <pose>-0.005 0.052 0.036 0 0 0</pose>
            <geometry>
            <box>
                <!-- size: x (width), y (thickness), z (height) -->
                <size>0.0085 0.003 0.0115</size>
            </box>
            </geometry>
            <material>
            <diffuse>1 1 1 1</diffuse>
            <ambient>0.8 0.8 0.8 1</ambient>
            </material>
        </visual>

        <visual name="tooth_right">
            <pose>0.005 0.052 0.036 0 0 0</pose>
            <geometry>
            <box>
                <size>0.0085 0.003 0.0115</size>
            </box>
            </geometry>
            <material>
            <diffuse>1 1 1 1</diffuse>
            <ambient>0.8 0.8 0.8 1</ambient>
            </material>
        </visual>
    </link>

    <!-- Prismatic joint (force-actuated via HardwareStation) -->
    <joint name="mole_slider" type="prismatic">
      <parent>socket</parent>
      <child>mole</child>
      <axis>
        <xyz>0 0 1</xyz>
        <limit>
          <lower>0.0</lower>
          <upper>0.2</upper>
          <effort>200</effort>
        </limit>
        <dynamics>
          <damping>5</damping>
        </dynamics>
      </axis>
    </joint>

    <static>false</static>
  </model>
</sdf>
    """
    with open(filename, "w") as f:
        f.write(sdf_content)

MOLE_SDF_PATH = Path('/datasets/_deepnote_work/assets/mole.sdf')
create_mole_sdf_file(MOLE_SDF_PATH)

In [10]:
from pathlib import Path
from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
from pydrake.all import (
    DiagramBuilder,
)

# HAMMER_SDF_PATH = Path(__file__).parent / "assets" / "hammer.sdf"
HAMMER_SDF_PATH = Path('/datasets/_deepnote_work/assets/hammer.sdf')
GRID_SDF_PATH = Path('/datasets/_deepnote_work/assets/grid.sdf')

MOLE_SCENE_YAML = f"""directives:

# ===============================================================
# IIWA
# ===============================================================
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

# ===============================================================
# Hammer
# ===============================================================
- add_model:
    name: hammer
    file: file://{HAMMER_SDF_PATH}
    default_free_body_pose:
      hammer_link:
        translation: [0, 0, 0]
        rotation: !Rpy {{ deg: [0, 0, 0] }}

- add_weld:
    parent: iiwa::iiwa_link_7
    child: hammer::hammer_link
    X_PC:
      translation: [0, 0, 0.06]
      rotation: !Rpy {{deg: [0, -90, 0] }}

# ===============================================================
# Grid Board + Moles
# ===============================================================
- add_model:
    name: grid_board
    file: file://{GRID_SDF_PATH}

- add_weld:
    parent: world
    child: grid_board::board
    X_PC:
      translation: [0, -0.75, 0]
      rotation: !Rpy {{ deg: [0, 0, 0] }}

# 3x3 mole grid
- add_model:
    name: mole_0_0
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_0_0::socket
    X_PC: {{translation: [-0.2, -0.2, 0.0125]}}

- add_model:
    name: mole_0_1
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_0_1::socket
    X_PC: {{translation: [0.0, -0.2, 0.0125]}}

- add_model:
    name: mole_0_2
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_0_2::socket
    X_PC: {{translation: [0.2, -0.2, 0.0125]}}

- add_model:
    name: mole_1_0
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_1_0::socket
    X_PC: {{translation: [-0.2, 0.0, 0.0125]}}

- add_model:
    name: mole_1_1
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_1_1::socket
    X_PC: {{translation: [0.0, 0.0, 0.0125]}}

- add_model:
    name: mole_1_2
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_1_2::socket
    X_PC: {{translation: [0.2, 0.0, 0.0125]}}

- add_model:
    name: mole_2_0
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_2_0::socket
    X_PC: {{translation: [-0.2, 0.2, 0.0125]}}

- add_model:
    name: mole_2_1
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_2_1::socket
    X_PC: {{translation: [0.0, 0.2, 0.0125]}}

- add_model:
    name: mole_2_2
    file: file://{MOLE_SDF_PATH}
- add_weld:
    parent: grid_board::board
    child: mole_2_2::socket
    X_PC: {{translation: [0.2, 0.2, 0.0125]}}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
    mole_0_0: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_0_1: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_0_2: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_1_0: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_1_1: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_1_2: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_2_0: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_2_1: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}
    mole_2_2: !InverseDynamicsDriver
      gains: 
        mole_slider: {{kp: 100, kd: 10}}

"""

In [ ]:
import numpy as np
from pydrake.systems.primitives import ConstantVectorSource

from pydrake.all import (
    DiagramBuilder, StartMeshcat, Simulator, MultibodyPlant, 
    Parser, AddMultibodyPlantSceneGraph, RigidTransform,
    LogVectorOutput, MeshcatVisualizer
)

def run_whack_a_mole():
    meshcat = StartMeshcat()
    builder = DiagramBuilder()
    
    scenario = LoadScenario(data=MOLE_SCENE_YAML)
    station = MakeHardwareStation(scenario, meshcat=meshcat)
    builder.AddSystem(station)

    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")

    iiwa_model = plant.GetModelInstanceByName("iiwa")
    mole_models = []
    for i in range(3):
        for j in range(3):
            mole_models.append(plant.GetModelInstanceByName(f"mole_{i}_{j}"))

    # Visualize axes (useful for debugging)
    # Get hammer head frame
    # hammer = plant.GetModelInstanceByName("hammer")
    # hammer_face_frame = plant.GetFrameByName("hammer_face", hammer)
    # AddFrameTriadIllustration(
    #     scene_graph=scene_graph,
    #     frame=hammer_face_frame,
    #     length=0.1,
    # )
        
    # ----- ADD SYTEMS -----
    
    # BonkBotBrain, aka the FSMPlanner
    planner = builder.AddSystem(BonkBotBrain(plant, iiwa_model, mole_models))
    planner.set_name("bonkbot_brain")
    
    # Perception: DummyPerception for now
    perception = builder.AddSystem(DummyPerception(plant, mole_models))
    perception.set_name("perception")

    # Force Sensor
    hammer_body = plant.GetBodyByName("hammer_link", plant.GetModelInstanceByName("hammer"))
    force_sensor = builder.AddSystem(HammerContactForce(plant, hammer_body.index(), [0,0,-1]))

    # Mole Controller
    mole_controller = builder.AddSystem(MoleController(plant, hammer_body.index(), 
                                        mole_models, switch_time_range=(1, 6), multi_mode=True, 
                                        random_switch_time=True))
    mole_controller.set_name("mole_controller")

    # ----- WIRING -----
    
    # Robot State -> Planner
    builder.Connect(station.GetOutputPort("iiwa.position_measured"), planner.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa.velocity_estimated"), planner.GetInputPort("iiwa_velocity"))

    # Plant State -> Perception
    builder.Connect(station.GetOutputPort("state"), perception.GetInputPort("plant_state"))

    # Station -> Force Sensor -> Planner
    builder.Connect(station.GetOutputPort("contact_results"), force_sensor.GetInputPort("contact_results"))
    builder.Connect(force_sensor.GetOutputPort("F_meas"), planner.GetInputPort("F_meas"))

    # Perception -> Planner
    builder.Connect(perception.GetOutputPort("mole_poses"), planner.GetInputPort("mole_poses"))

    # Planner -> Robot
    builder.Connect(planner.GetOutputPort("iiwa_position_command"),station.GetInputPort("iiwa.position"))

    # Game Controller -> Station
    n = 0
    for i in range(3):
        for j in range(3):
            builder.Connect(
                mole_controller.GetOutputPort(f"mole_{n}_setpoint"),
                station.GetInputPort(f"mole_{i}_{j}.desired_state")
            )
            n += 1
        
    # Station -> Game Controller
    builder.Connect(station.GetOutputPort("contact_results"), mole_controller.GetInputPort("contact_results"))
    builder.Connect(station.GetOutputPort("state"), mole_controller.GetInputPort("plant_state"))

    # ----- VISUALIZATION -----
    diagram = builder.Build()
    # RenderDiagram(diagram, max_depth=1)
    simulator = Simulator(diagram)

    # Run Simulation
    simulator.set_target_realtime_rate(1.0)
    meshcat.AddButton("Stop Simulation", "Escape")
    meshcat.StartRecording()
    print("Starting simulation... Mole 0 is UP.")
    dt = 0.2    
    while meshcat.GetButtonClicks("Stop Simulation") < 1:
        context = simulator.get_context()
        simulator.AdvanceTo(context.get_time() + dt)
    print("Stopping simulation and finalizing recording...")
    meshcat.StopRecording()
    meshcat.PublishRecording()
    meshcat.DeleteButton("Stop Simulation")
    print("Done.")


run_whack_a_mole()

INFO:drake:Meshcat listening for connections at https://e09b4ee5-e97d-4566-8819-f89ba4350a2e.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/7b92aacbe021861ec9bbbb82d8ab9a19ded970ff.tar.gz
Starting simulation... Mole 0 is UP.
[DEBUG t=0.00] State: 0
[Brain] No mole is up.
[DEBUG t=1.00] State: 0
[Brain] No mole is up.
[DEBUG t=2.00] State: 0
[Brain] No mole is up.
[DEBUG t=3.00] State: 0
[Brain] No mole is up.
[DEBUG t=4.00] State: 0
[Brain] No mole is up.
[DEBUG t=5.00] State: 0
[Brain] No mole is up.
[Mole] Mole 0 -> UP (Multi Mode)
[DEBUG t=6.00] State: 0
[Brain] No mole is up.
[Brain] DETECTED MOLE 0! ATTACK! Switch to PLANNING.
[Brain] Planning pre-hit path to mole 0 at [-0.2        -0.95        0.10288239]
[Brain] Prehit path found. Switch to APPROACH.
[DEBUG t=7.00] State: 2
[Brain] Approaching... (0.96/2.00s)
[DEBUG t=8.00] State: 2
[Brain] Approaching... (1.96/2.00s)
[Brain] Pre-hit reached. Switching to ADMITTANCE HIT.
[BONK] Mole 0 bonked! (Height: 0.021m)
[Brain] Hit timeout/complete. Switch to RECOVER.
[DEBUG t=9.00] State: 4
[

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e09b4ee5-e97d-4566-8819-f89ba4350a2e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>